In [1]:
import os
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Thiết lập các tùy chọn cho Chrome
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless=new") 
chrome_options.add_argument("start-maximized")  # Mở trình duyệt ở chế độ toàn màn hình
chrome_options.add_argument("--disable-dev-shm-usage")  # Giảm thiểu lỗi bộ nhớ trên Linux
chrome_options.add_argument("--no-sandbox")  # Giảm thiểu lỗi bộ nhớ trên Linux


In [3]:
# Khởi tạo đối tượng driver cho Chrome
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),  # Tự động tải và cài đặt chromedriver
    options=chrome_options
)

In [4]:
root_dir = './mimikara_n2'
os.makedirs(root_dir, exist_ok=True)

In [5]:
def get_element_text(data, class_name):
    try:
        return data.find_element(By.CLASS_NAME, class_name).text.strip()
    except:
        return None


In [6]:
lessons_per_unit = {
    1: 6,
    2: 9,
    3: 5,
    4: 12,
    5: 5,
    6: 5,
    7: 8,
    8: 9,
    9: 4,
    10: 4,
    11: 8,
    12: 9,
    13: 5
}

for unit, num_lessons in lessons_per_unit.items():
    for lesson in range(1, num_lessons + 1):
        # Tạo URL cho từng bài học
        url = f'https://www.vnjpclub.com/mimi-kara-n2-goi/unit-{unit}-bai-{lesson}.html'
        driver.get(url)
        time.sleep(1)

        # Đợi cho phần tử cần lấy xuất hiện
        try:
            wait = WebDriverWait(driver, 10)
            box = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div/div/div/div/div[1]/div[1]/div/div/article/div')))
        except Exception as e:
            print(f"Không tìm thấy nội dung cho Unit {unit}, Bài {lesson}: {e}")
            continue

        # Lấy danh sách các từ vựng
        datas = box.find_elements(By.CLASS_NAME, 'boxtv')

        # Tạo file CSV để lưu nội dung
        filename = os.path.join(root_dir, f'mimikara_unit{unit:02d}_lesson{lesson:02d}.csv')
        with open(filename, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            # Ghi dòng tiêu đề
            writer.writerow(['Kanji', 'Furigana', 'Hán Việt', 'Nghĩa', 'Ví dụ', 'Nghĩa Ví dụ'])

            for data in datas:
                data.click()
                
                # Lấy phần tử `ruby` và nội dung kanji
                try:
                    kanji = data.find_element(By.XPATH, ".//div[@class='tuvung']//b")
                    kanji_raw = kanji.text.replace('\n', '').strip()

                    # Tách kanji và furigana
                    try:
                        ruby_element = data.find_element(By.TAG_NAME, "ruby")
                        rt_text = kanji.find_element(By.TAG_NAME, "rt").text.strip()
                        kanji_text = kanji_raw.replace(rt_text, "").strip()
                        furigana = ruby_element.find_element(By.TAG_NAME, "rt").text.strip()
                        kanji_only = kanji_text.replace(furigana, "").strip()
                        kanji_alone = ruby_element.text.strip().replace(furigana, "").strip()
                        kana = kanji_raw.replace(kanji_alone, "").strip()
                    except:
                        kanji_only = kanji_raw.replace(kanji_alone, "").strip()


                    # Tách kanji và furigana
                    try:
                        furigana = ruby_element.find_element(By.TAG_NAME, "rt").text.strip()
                        kanji_alone = ruby_element.text.strip().replace(furigana, "").strip()
                        kana = kanji_raw.replace(kanji_alone, "").strip()
                    except:
                        kana = "-"

                    # Lấy nghĩa và ví dụ
                    hanviet = get_element_text(data, 'hanviet1') or get_element_text(data, 'hanviet2') or get_element_text(data, 'hanviet') or get_element_text(data, 'hanviet3')
                    meaning = get_element_text(data, 'nghia1') or get_element_text(data, 'nghia2') or get_element_text(data, 'nghia') or get_element_text(data, 'nghia3')

                    vidubox = data.find_element(By.CLASS_NAME, "vidubox")
                    nghiavidu = vidubox.find_element(By.CLASS_NAME, "nghiavidu").text.strip()

                    example_element = vidubox.find_element(By.CLASS_NAME, "tuvung")
                    clean_text = example_element.text.replace('\n', '')

                    # Xử lý chuỗi ví dụ
                    stt = example_element.find_element(By.CLASS_NAME, "viduseq").text.strip()
                    clean_text = clean_text.replace(stt, "").strip()
                    clean_text = clean_text[:-1]
                    clean_text = clean_text.replace("「", "").replace("」", "")

                    # Lấy tất cả các thẻ ruby bên trong phần tử ví dụ
                    ruby_elements = example_element.find_elements(By.TAG_NAME, "ruby")
                    for ruby in ruby_elements:
                        furi = ruby.find_element(By.TAG_NAME, "rt").text.strip()
                        clean_text = clean_text.replace(furi, "").strip()

                    # Ghi dữ liệu vào file CSV
                    writer.writerow([kanji_only, kana, hanviet, meaning, clean_text, nghiavidu])

                except Exception as e:
                    print(f"Lỗi khi lấy dữ liệu từ bài học Unit {unit}, Bài {lesson}: {e}")
                    continue

        print(f"Đã lưu nội dung vào file CSV: {filename}")
# Đóng trình duyệt
driver.quit()


Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson01.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson02.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson03.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson04.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson05.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit01_lesson06.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson01.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson02.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson03.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson04.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson05.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson06.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikara_unit02_lesson07.csv
Đã lưu nội dung vào file CSV: ./mimikara_n2\mimikar